In [2]:
import pandas as pd
import zipfile
import numpy as np

### A Fazeres
- Importar dados do IPCA <br>
Precisa converter datas em datetime
- Adicionar dados trimestrais e todos os indicadores imobiliários disponíveis neles

- Tabelas descritivas dos dados, gráficos, etc...
- Descrever as variáveis disponíveis nos dados e destacar as mais importantes



Referência:<br>
Dados FIIs - https://dados.cvm.gov.br/dataset/fii-doc-inf_mensal<br>
IPCA - https://www.ibge.gov.br/estatisticas/economicas/precos-e-custos/9256-indice-nacional-de-precos-ao-consumidor-amplo.html?=&t=series-historicas<br>
CDI - https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-de-segmentos-e-setoriais/serie-historica-do-di.htm <br>
Benchmarks - Economatica<br>

### Problemas:

- Preços estam problemáticos / Total Return problemático
- Dados de fechamento não coincidem com realizado, mesmo se tratando dos preços sem ajuste de proventos
- Base CNPJ/Tickers tem dados faltantes

Importando os dados FIIs

In [3]:
start_date = 2016
end_date = 2024

diretorio = r"C:\Users\Gustavo Aurelio\Desktop\TCC\bases_zip"
base_mensal = pd.DataFrame()

for year in range(start_date, end_date+1):
    zip_mensal = zipfile.ZipFile(diretorio+ f"\Mensal_zip\inf_mensal_fii_{year}.zip")
    # zip_trimestral = zipfile.ZipFile(diretorio + f"\Trimestral_zip\inf_trimestral_fii_{year}.zip")

    bp_inf_mensal = pd.read_csv(zip_mensal.open(f'inf_mensal_fii_ativo_passivo_{year}.csv'), delimiter=";",encoding='latin-1')
    complem_inf_mensal = pd.read_csv(zip_mensal.open(f'inf_mensal_fii_complemento_{year}.csv'), delimiter=";", encoding='latin-1')
    geral_inf_mensal = pd.read_csv(zip_mensal.open(f'inf_mensal_fii_geral_{year}.csv'), delimiter=";", encoding='latin-1')
    base_mensal_year=bp_inf_mensal.merge(complem_inf_mensal,how='outer',on=['Data_Referencia','CNPJ_Fundo','Versao'])
    base_mensal_year=base_mensal_year.merge(geral_inf_mensal,how='outer',on=['Data_Referencia','CNPJ_Fundo','Versao'])

    base_mensal = pd.concat([base_mensal, base_mensal_year], ignore_index=True)

    #Precisa ajustar base trimestral
    # imovel_inf_trimestral = pd.read_csv(zip_trimestral.open(f'inf_trimestral_fii_imovel_{year}.csv'), delimiter=";",encoding='latin-1')
    # result_inf_trimestral = pd.read_csv(zip_trimestral.open(f'inf_trimestral_fii_resultado_contabil_financeiro_{year}.csv'), delimiter=";", encoding='latin-1')

base_mensal['Data_Referencia'] = pd.to_datetime(base_mensal.Data_Referencia)
cnpj_ticker = pd.read_excel('CNPJ e TICKER FIIs.xlsx',sheet_name='FIIs',header=6)
fech_nao_ajust = pd.read_csv(r'C:\Users\Gustavo Aurelio\Desktop\TCC\bases_gerais\economatica_fech_nao_ajust.csv',delimiter=';',header=3)

Importando dados auxiliares

In [ ]:
cdi = pd.read_csv(r'C:\Users\Gustavo Aurelio\Desktop\TCC\bases_gerais\cdi.csv', delimiter=";", header = 38 )
cdi = cdi.replace({',':'.',
                    '-':np.nan,
                    'ND':np.nan},regex=True)
cdi['Taxa SELIC'] = pd.to_numeric(cdi['Taxa SELIC'])
cdi['Data'] = pd.to_datetime(cdi.Data, dayfirst=True)
cdi.columns = ['Data', 'tx_selic']

ipca = pd.read_csv(r'C:\Users\Gustavo Aurelio\Desktop\TCC\bases_gerais\ipca.csv', delimiter=";", header = 1 )

benchmarks = pd.read_csv(r'C:\Users\Gustavo Aurelio\Desktop\TCC\bases_gerais\economatica_benchmarks.csv', delimiter=";", header = 3 )
benchmarks.columns = benchmarks.columns.str.replace('Fechamento\najust p/ prov\nEm moeda orig\n','', regex = True)
benchmarks = benchmarks.replace({',':'.',
                        '-':np.nan},regex=True)
benchmarks['Data'] = pd.to_datetime(benchmarks.Data, dayfirst=True)

Cleaning and adjusting database

In [4]:
#Cleaning no ajustments close price
fech_nao_ajust.columns = fech_nao_ajust.columns.str.replace('Fechamento\nnão aj p/ prov\nEm moeda orig\n','',regex = True)
fech_nao_ajust = fech_nao_ajust.replace({',':'.',
                        '-':np.nan,
                        'Fechamento\nnão aj p/ prov\nEm moeda orig\n':''},regex=True)
fech_nao_ajust['Data'] = pd.to_datetime(fech_nao_ajust.Data,dayfirst=True)

datas_desde_2003 = pd.date_range(start='2003-01-01', end=pd.Timestamp.today(), freq='D')
df_datas = pd.DataFrame({'Data': datas_desde_2003})
fech_nao_ajust = pd.merge(df_datas, fech_nao_ajust, on='Data', how='left')
fech_nao_ajust = fech_nao_ajust.ffill()

fech_nao_ajust = fech_nao_ajust.set_index('Data')
fech_nao_ajust = fech_nao_ajust.apply(pd.to_numeric, errors='ignore').reset_index()
fech_nao_ajust = fech_nao_ajust.ffill() #preencher valores faltantes com ultimo valor disponível, nos casos de final de semana ou feriados terem dados de referência

fech_tidy = fech_nao_ajust.melt(id_vars='Data')
fech_tidy.columns = ['Data_Referencia', 'Ticker', 'Fechamento']

#Cleaning monthly database
base_mensal['CNPJ_Fundo'] = base_mensal['CNPJ_Fundo'].str.replace('/','',regex=False)
base_mensal['CNPJ_Fundo'] = base_mensal['CNPJ_Fundo'].str.replace('.','',regex=False)
base_mensal['CNPJ_Fundo'] = base_mensal['CNPJ_Fundo'].str.replace('-','',regex=False)

#Pareando CNPJ e Tickers
cnpj_ticker = (cnpj_ticker.loc[:,['Ticker','CNPJ/Fundo']].drop_duplicates()).dropna()
cnpj_ticker_dic = dict(zip(cnpj_ticker['CNPJ/Fundo'], cnpj_ticker['Ticker']))
base_mensal['Ticker'] = base_mensal['CNPJ_Fundo'].map(cnpj_ticker_dic)

In [86]:
indicadores = pd.DataFrame()
indicadores['CNPJ_Fundo'] = base_mensal['CNPJ_Fundo']
indicadores['Data_Referencia'] = base_mensal['Data_Referencia']
indicadores['Versao'] = base_mensal['Versao']
indicadores['Ticker'] = base_mensal['Ticker']
indicadores = indicadores.merge(fech_tidy,left_on = ['Ticker','Data_Referencia'], right_on = ['Ticker', 'Data_Referencia'], how= 'left')
indicadores['Patrimonio_liquido'] = base_mensal.Patrimonio_Liquido
indicadores['Total_Ativo'] = base_mensal.Valor_Ativo
indicadores['Total_Passivo'] = base_mensal.Total_Passivo
indicadores['Disponibilidades'] = base_mensal.Disponibilidades
indicadores['Rend_distrib'] = base_mensal.Rendimentos_Distribuir
indicadores['n_cotas'] = base_mensal.Cotas_Emitidas
indicadores['Valor_patri_cotas'] = base_mensal.Valor_Patrimonial_Cotas
indicadores['Div_p_cota'] = indicadores.Rend_distrib/indicadores.n_cotas
# Percentual_Dividend_Yield_Mes

indicadores['Valor_cx_perc'] = indicadores.Disponibilidades/indicadores.Total_Ativo
indicadores['Alavancagem'] = indicadores.Total_Passivo/indicadores.Patrimonio_liquido
indicadores['P_vp'] = indicadores.Fechamento/indicadores.Valor_patri_cotas
indicadores['Div_yeld'] = indicadores.Div_p_cota/indicadores.Fechamento
indicadores['conferir_dymes'] = base_mensal.Percentual_Dividend_Yield_Mes

#Dividendos 12 Meses
Dividendos = indicadores[['Data_Referencia','Ticker','Rend_distrib']].set_index('Data_Referencia')
Dividendos = Dividendos.pivot_table(index='Data_Referencia',columns='Ticker',values='Rend_distrib')
Dividendos_12 = Dividendos.rolling(12).sum().reset_index().melt(id_vars='Data_Referencia')
Dividendos_12.columns = ['Data_Referencia', 'Ticker', 'Dividendos_12']
indicadores = indicadores.merge(Dividendos_12,left_on = ['Ticker','Data_Referencia'], right_on = ['Ticker', 'Data_Referencia'], how= 'left')

indicadores['Dividend_yield_12'] = (indicadores.Dividendos_12/indicadores.n_cotas)/indicadores.Fechamento

In [87]:
#Teste
indicadores[indicadores.Ticker == 'KNSC11']

,CNPJ_Fundo,Data_Referencia,Versao,Ticker,Fechamento,Patrimonio_liquido,Total_Ativo,Total_Passivo,Disponibilidades,Rend_distrib,n_cotas,Valor_patri_cotas,Div_p_cota,Valor_cx_perc,Alavancagem,P_vp,Div_yeld,conferir_dymes,Dividendos_12,Dividend_yield_12
19264,35864448000138,2020-10-01,1,KNSC11,NaN,2.523750e+08,2.526067e+08,2.316625e+05,88678.54,NaN,2632885.0,95.854926,NaN,0.000351,0.000918,NaN,NaN,0.000000,NaN,NaN
19265,35864448000138,2020-11-01,1,KNSC11,NaN,2.541525e+08,2.558849e+08,1.732429e+06,23792.13,1263784.80,2632885.0,96.530025,0.480000,0.000093,0.006816,NaN,NaN,0.005008,NaN,NaN
19266,35864448000138,2020-12-01,1,KNSC11,96.00,2.590605e+08,2.618808e+08,2.820251e+06,74101.40,2316938.80,2632885.0,98.394167,0.880000,0.000283,0.010886,0.975668,0.009167,0.009116,NaN,NaN
25677,35864448000138,2021-01-01,1,KNSC11,99.01,2.539809e+08,2.578324e+08,3.851525e+06,876971.70,3370092.80,2632885.0,96.464871,1.280000,0.003401,0.015165,1.026384,0.012928,0.013009,NaN,NaN
25678,35864448000138,2021-02-01,1,KNSC11,102.59,2.479546e+08,2.511272e+08,3.172667e+06,28136.51,2843515.80,2632885.0,94.175995,1.080000,0.000112,0.012795,1.089343,0.010527,0.011196,NaN,NaN
25679,35864448000138,2021-03-01,1,KNSC11,105.35,2.469337e+08,2.497230e+08,2.789292e+06,24600.41,2395925.35,2632885.0,93.788255,0.910000,0.000099,0.011296,1.123275,0.008638,0.009663,NaN,NaN
25680,35864448000138,2021-04-01,1,KNSC11,106.18,2.516373e+08,2.775238e+08,2.588649e+07,21024.81,2448583.05,2632885.0,95.574749,0.930000,0.000076,0.102872,1.110963,0.008759,0.009916,NaN,NaN
25681,35864448000138,2021-05-01,1,KNSC11,101.93,6.133193e+08,6.175774e+08,4.258092e+06,20798.23,3219867.67,6429953.0,95.384729,0.500761,0.000034,0.006943,1.068620,0.004913,0.005239,NaN,NaN
25682,35864448000138,2021-06-01,1,KNSC11,101.75,6.090382e+08,6.159542e+08,6.915954e+06,20119.33,6108455.35,6429953.0,94.718920,0.950000,0.000033,0.011356,1.074231,0.009337,0.009960,NaN,NaN
25683,35864448000138,2021-07-01,1,KNSC11,101.18,6.048755e+08,6.636467e+08,5.877123e+07,20766.61,6429953.00,6429953.0,94.071528,1.000000,0.000031,0.097163,1.075565,0.009883,0.010558,NaN,NaN


Adicionar Dados Trimestrais

P/M²

Aluguel/M²

PL/M²